<a href="https://colab.research.google.com/github/abulhasanat/MachineLearning/blob/master/Web_classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install JPype1

     |████████████████████████████████| 2.7MB 3.1MB/s 


In [2]:
!git clone https://github.com/misja/python-boilerpipe.git

Cloning into 'python-boilerpipe'...
remote: Enumerating objects: 264, done.
remote: Total 264 (delta 0), reused 0 (delta 0), pack-reused 264
Receiving objects: 100% (264/264), 30.28 KiB | 2.75 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [3]:
%cd /content/python-boilerpipe
!sudo python setup.py install

/content/python-boilerpipe
/usr/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'install_requires'
  warnings.warn(msg)
running install
running build
running build_py
creating build
creating build/lib
creating build/lib/boilerpipe
copying src/boilerpipe/__init__.py -> build/lib/boilerpipe
creating build/lib/boilerpipe/extract
copying src/boilerpipe/extract/__init__.py -> build/lib/boilerpipe/extract
creating build/lib/boilerpipe/data
creating build/lib/boilerpipe/data/boilerpipe-1.2.0
copying src/boilerpipe/data/boilerpipe-1.2.0/boilerpipe-1.2.0.jar -> build/lib/boilerpipe/data/boilerpipe-1.2.0
creating build/lib/boilerpipe/data/boilerpipe-1.2.0/lib
copying src/boilerpipe/data/boilerpipe-1.2.0/lib/xerces-2.9.1.jar -> build/lib/boilerpipe/data/boilerpipe-1.2.0/lib
copying src/boilerpipe/data/boilerpipe-1.2.0/lib/nekohtml-1.9.13.jar -> build/lib/boilerpipe/data/boilerpipe-1.2.0/lib
running install_lib
creating /usr/local/lib/python3.6/dist-packages/boilerpi

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import numpy as np
import re
from urllib.parse import urlparse

In [6]:
import requests
requests.packages.urllib3.disable_warnings()

import random
import math
import string
import re
import numpy as np
import nltk
nltk.download('punkt')
import networkx as nx
import json

import collections
from boilerpipe.extract import Extractor

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [0]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [0]:
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [9]:
%%bash
pip install pytorch-pretrained-bert

  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609182 sha256=9e3fcb95f580c54c606f50f5e1c59dc5440e0187da552941bef8dc224fdb75a8
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built regex


In [0]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=True,
    bert_model_name="bert-base-uncased",
    max_lr=3e-5,
    epochs=4,
    use_fp16=False,
    bs=32,
    discriminative=False,
    max_seq_len=256,
)

In [11]:
from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    config.bert_model_name,
)

100%|██████████| 231508/231508 [00:00<00:00, 5897162.18B/s]


In [0]:
def _join_texts(texts:Collection[str], mark_fields:bool=False, sos_token:Optional[str]=BOS):
    """Borrowed from fast.ai source"""
    if not isinstance(texts, np.ndarray): texts = np.array(texts)
    if is1d(texts): texts = texts[:,None]
    df = pd.DataFrame({i:texts[:,i] for i in range(texts.shape[1])})
    text_col = f'{FLD} {1} ' + df[0].astype(str) if mark_fields else df[0].astype(str)
    if sos_token is not None: text_col = f"{sos_token} " + text_col
    for i in range(1,len(df.columns)):
        #text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i]
        text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i].astype(str)
    return text_col.values

In [0]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [0]:
class BertTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class BertNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=Vocab(list(bert_tok.vocab.keys())), **kwargs)

def get_bert_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for BERT
    We remove sos/eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original BERT model.
    """
    return [BertTokenizeProcessor(tokenizer=tokenizer),NumericalizeProcessor(vocab=vocab)]

In [0]:
def getUA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]

    return random.choice(uastrings)

def prepare_sentences(text):
    text=text.replace('\n',' ')
    # Clean characters
    text = "".join([t for t in text if t.isalnum() or t in string.punctuation or t == ' '])
    
    # Get rid of whitespace characters
    sentences = nltk.sent_tokenize(text)
    text = " ".join([s.strip() for s in sentences])
    
    # Fix puctuation spacing
    text = re.sub(r"(\w{2,})([\.\!\?]+)(\w)", r"\1\2 \3", text)
    text = re.sub(' +', ' ', text)
    return text

def find_children(n1, n2):
    
    children1 = [c for c in n1.children]
    children2 = [c for c in n2.children]
    
    for i, children in enumerate([children1,children2]):
        level = 1

        while children:
            lchildren = children
            children = []
            for ch in lchildren:
                if not ch.name:
                    continue
                if ch == n1 or ch == n2:
                    if i == 0:
                        return len( nltk.sent_tokenize(n2.get_text()) ) * ( 1/math.exp(level) )
                    else:
                        return len( nltk.sent_tokenize(n1.get_text()) ) * ( 1/math.exp(level) )
                                        
                children.extend(ch.children)

            level += 1
            
    return 0

def extract_html(url, timeout=10):
    headers = {'user-agent': getUA()}
    r = requests.get(url, headers = headers, verify=False, timeout=timeout)
    html = r.content
    return html

def tokenize_url(url:str):   
    url=re.sub("(\W|_)+"," ",url)
    return url

In [0]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [0]:
def get_title(soup):
    try:
        for i in list(soup.find_all('title')):
            return(i.get_text().replace("\n"," ").replace("\t"," ").strip())
    except:
        pass

def list2string(x):
    s=""
    try:
        for i in x:
            for j in i:
                if j!=None:
                    s = s + j+" "
    except:
        pass
    return (s)

def get_metadata_property(soup):
    try:
        x=[i.get('property') for i in soup.findAll('meta')]
        v=[i.get('content') for i in soup.findAll('meta')]
        z=np.c_[x,v]
        return list2string(z)
    except:
        pass

def get_metadata_name(soup):
    try:
        y=[i.get('name') for i in soup.findAll('meta')]
        v=[i.get('content') for i in soup.findAll('meta')]
        z=np.c_[y,v]
        return list2string(z)
    except:
        pass

def get_metadata_content(soup):
    try:
        v=[]
        v=[i.get('content') for i in soup.findAll('meta')]
    #     return list2string(v)
        return (v)
    except:
        pass

def get_text(soup):
    text=""
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            text+=item.get_text()+" "
    except:
        pass
    return text
#     return (len(text.split())/COUNT)

In [0]:
def get_word_count(soup):
    text=""
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            text+=item.get_text()+" "
    except:
        pass
    return len(text.split())

def get_para_count(soup):
    count=0
    try:
        for item in list(soup.find_all('p')):  #[:2]:
            count+=1
    except:
        pass
    return count

In [0]:
def get_internal_links(url_main,links):
    internal_url_list=[]
    try:
        for item in links:
            for url_parts in url_main.split('.')[:-1]:
                if (url_parts in item.lower()):
                    internal_url_list.append(item)
        internal_url_list=list(dict.fromkeys(internal_url_list))    
        return(len(internal_url_list))
    except:
        pass

def get_external_links(url_main,links):
    external_url_list=[]
    try:
        for item in links:
            count=0
            for url_parts in url_main.split('.')[:-1]:
                if (item.startswith(('http', 'ftp', 'www')) and (url_parts not in item.lower())):
                    count+=1
            if count==len(url_main.split('.')[:-1]):
                external_url_list.append(item)
        return(len(external_url_list))
    except:
        pass

In [0]:
def extract_features(url):
    # create a new Chrome session
    try:
        print(url)
        metadata_property=''
        metadata_name=''
        metadata_content=''
        title=''
        word_count=0
        sen_count=0
        internal_links=0
        external_links=0
    #         driver = webdriver.Chrome(executable_path=r'D:/chromedriver_win32/chromedriver.exe')
    #         driver.set_page_load_timeout(60)
    #         driver.get(url)
    #         html = driver.page_source
    #         driver.close()
    #         soup = BeautifulSoup(html, 'html.parser')  #Parse html code       
    #         body = soup.find('body')

        html= extract_html(url, 100)
        soup = BeautifulSoup(html, 'lxml')
        body = soup.find('body')
        remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
                                             "svg", "h1","h2", "h3", "h4", "h5", "xml"])]
        text = ""
        boilerpipe_sentences=""
        boilerpipe_content = prepare_sentences(Extractor(extractor='CanolaExtractor', html=html).getText())
        boilerpipe_sentences = prepare_sentences(boilerpipe_content)
    #         print('Sentences:',len(boilerpipe_sentences))

#         p = '(?:http.*://)?(?:www.)?(?P<host>[^/ ]+).?(?P<port>[0-9]*).*'
        p = '(?:http.*://)?(?:www[0-9].|www.)?(?P<host>[^/ ]+).?(?P<port>[0-9]*).*'
        m = re.search(p,url)
        url_main = m.group('host') # 'www.abc.com'
        if len(soup)>0:
            metadata_property=get_metadata_property(soup)
            metadata_name=get_metadata_name(soup)
            metadata_content=get_metadata_content(soup)
            title=get_title(soup)
            
            word_count=len(nltk.word_tokenize(boilerpipe_sentences))
            
            sen_count=len(nltk.sent_tokenize(boilerpipe_sentences))       

            links = [ x['href'] for x in soup.find_all('a', href=True) ]
            if len(links)>0:
                internal_links=get_internal_links(url_main,links)
                external_links=get_external_links(url_main,links)

    #         return pd.Series([get_metadata_property(soup),get_metadata_name(soup),get_metadata_content(soup),
    #                       get_title(soup)])
    except:
        pass
    
    extract= pd.Series([metadata_property,metadata_name,metadata_content,
                          title,word_count,sen_count,internal_links,external_links])
    #         extract= pd.Series([get_metadata_property(soup),get_metadata_name(soup)])
    return extract

In [0]:
# test=pd.DataFrame(pd.Series([boilerpipe_sentences,0,0,0,0,0,0]),columns=['category','label_1',	'label_3',	'label_5',	'label_6',	'label_7',	'label_8'])

In [0]:
# label_cols=['label_1',	'label_3',	'label_5',	'label_6',	'label_7',	'label_8']

In [43]:
DATA_ROOT=Path('/content/drive/My Drive/Data/DMOZ')
# learner.load(file=DATA_ROOT/'Model_DMOZ(6_cat).pkl')
learn = load_learner(DATA_ROOT, 'Model_DMOZ(6_cat).pkl')
# learn = load_learner(DATA_ROOT, 'Model_DMOZ(25_cat).pkl')
# learn = load_learner(DATA_ROOT, 'Model_DMOZ(all_cat_v1_2).pkl')
learn.to_fp32()


Learner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: MultiCategoryList

Path: /content/drive/My Drive/Data/DMOZ;

Valid: LabelList (0 items)
x: TextList

y: MultiCategoryList

Path: /content/drive/My Drive/Data/DMOZ;

Test: None, model=BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): 

In [53]:
url='https://docs.fast.ai/text.html'
html= extract_html(url, 60)
soup = BeautifulSoup(html, 'lxml')
body = soup.find('body')
# remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
#                                      "svg", "h1","h2", "h3", "h4", "h5", "xml"])]
remove = [s.extract() for s in body(["script", "style","iframe","noscript","nav","footer","header",\
                                     "svg", "xml"])]
boilerpipe_content = prepare_sentences(Extractor(extractor='CanolaExtractor', html=html).getText())
boilerpipe_sentences = prepare_sentences(boilerpipe_content)
print(boilerpipe_sentences)

The text module of the fastai library contains all the necessary functions to define a Dataset suitable for the various NLP (Natural Language Processing) tasks and quickly generate models you can use for them. Specifically: text. transform contains all the scripts to preprocess your data, from raw text to token ids, text. data contains the definition of TextDataBunch , which the main class you'll need in NLP, text. learner contains helper functions to quickly create a language model or an RNN classifier. Have a look at the links above for full details of the API of each module, of read on for a quick overview. Quick Start: Training an IMDb sentiment model with ULMFiT Let's start with a quick end-to-end example of training a model. We'll train a sentiment classifier on a sample of the popular IMDb data, showing 4 steps: from fastai. text import * Contrary to images in Computer Vision, text can't directly be transformed into numbers to be fed into a model. The first thing we need to do i

In [54]:
learn.predict(boilerpipe_sentences)

(MultiCategory label_5,
 tensor([0., 0., 1., 0., 0., 0.]),
 tensor([0.0609, 0.1144, 0.8853, 0.0432, 0.0450, 0.0558]))